# **IMPORT LIBRARY**

In [29]:
import cv2
import os
import re
import math
import numpy as np
from ultralytics import YOLO
import pytesseract
from PIL import Image

# **CONFIG TESTING**

In [ ]:
# --- 0. Konfigurasi ---

# Sesuaikan dengan model path yang dimiliki
model_path = 'my_model.pt'
# Sesuikan dengan image path yang ingin di test
image_path = 'image_path'

output_crop_dir = "cropped_objects" 
CONF_THRESHOLD = 0.5

# Pastikan direktori output ada
os.makedirs(output_crop_dir, exist_ok=True)

# **DATA LOADING**

In [44]:
# Image Loading
try:
    model = YOLO(model_path, task='obb') # Tentukan task='obb' secara eksplisit
except Exception as e:
    print(f"Gagal memuat model: {e}")
    exit()

# Muat gambar dan ubah ukuran
original_image = cv2.imread(image_path)
if original_image is None:
    print(f"Gagal memuat gambar dari: {image_path}")
    exit()

# Gambar yang di-resize harus dalam BGR (default CV2) agar sesuai dengan hasil BBox
resized_image = cv2.resize(original_image, (640, 640), interpolation=cv2.INTER_LINEAR)

# **PREDICTION**

In [ ]:

# Model Prediction
results = model.predict(
    source=resized_image, 
    conf=CONF_THRESHOLD, 
    save=False,
    verbose=True # Matikan output log yang berlebihan
)

--- Inferensi Dimulai pada /Users/rio/Downloads/iLoveIMG Converted/IMG_1640.jpg ---

0: 640x640 1 kalori, 1 lemak, 1 nutrition-fact, 1 serving, 1 takaran-satuan, 247.9ms
Speed: 8.7ms preprocess, 247.9ms inference, 14.5ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
# Dictionary ini akan menyimpan deteksi terbaik untuk setiap label kelas
best_detections = {}

for r in results:
    # Mengakses hasil OBB
    # Gunakan r.boxes untuk kotak batas persegi standar (lebih mudah untuk cropping)
    boxes = r.boxes if r.boxes is not None and len(r.boxes) > 0 else (r.obb if hasattr(r, 'obb') and r.obb is not None else None)

    # Perbaikan: Cek apakah ada deteksi yang valid
    if boxes is None or len(boxes) == 0:
        continue

    # Filter Object duplikat
    for box in boxes:
        conf = box.conf[0].item()
        cls = box.cls[0].item()
        label = model.names.get(int(cls), f"Unknown_{int(cls)}")

        # tidak membutuhkan object nutrition-fact
        if label == "nutrition-fact":
            continue

        # menyimpan hasil deteksi
        # Kunci (key) adalah label, Nilai (value) adalah dictionary berisi box dan conf
        if label not in best_detections or conf > best_detections[label]['conf']:
            best_detections[label] = {
                'box': box,
                'conf': conf,
                'cls': cls
            }

# **CROPPING**

In [ ]:
print("\n--- Proses Cropping Dimulai ---")
detection_count = 0

# Iterasi Croping
for label, data in best_detections.items():
    
    # Ambil data dari deteksi terbaik
    box = data['box']
    conf = data['conf']
    cls = data['cls']
    
    # Mendapatkan BBox dan Label
    # Koordinat dalam bentuk (x1, y1, x2, y2)
    xyxy_tensor = box.xyxy[0].cpu() # Ambil tensor, pastikan di CPU
    x_min, y_min, x_max, y_max = xyxy_tensor.numpy().astype(int)

    # Melakukan cropping dengan Slicing NumPy (CV2)
    # Format Slicing: [y_min:y_max, x_min:x_max]
    cropped_segment = resized_image[y_min:y_max, x_min:x_max]

    # Chekck and save image
    # Minimal lebar atau tinggi 1 piksel
    if cropped_segment.shape[0] > 1 and cropped_segment.shape[1] > 1:
        # Nama file: label_index_conf.jpg
        output_filename = os.path.join(output_crop_dir, f"{label}_{detection_count:02d}_{int(conf*100)}.jpg")
        cv2.imwrite(output_filename, cropped_segment)
        
        print(f"✅ Berhasil crop '{label}' (Conf: {conf:.2f}). Disimpan di: {output_filename}")
        detection_count += 1
    else:
        print(f"❌ Gagal crop '{label}'. BBox sangat kecil atau tidak valid.")

print(f"\nProses selesai. Total {detection_count} objek unik telah di-crop dan disimpan di direktori '{output_crop_dir}'.")


--- Proses Cropping Dimulai ---
✅ Berhasil crop 'takaran-satuan' (Conf: 0.88). Disimpan di: cropped_objects/takaran-satuan_00_88.jpg
✅ Berhasil crop 'serving' (Conf: 0.68). Disimpan di: cropped_objects/serving_01_68.jpg
✅ Berhasil crop 'kalori' (Conf: 0.67). Disimpan di: cropped_objects/kalori_02_66.jpg
✅ Berhasil crop 'lemak' (Conf: 0.58). Disimpan di: cropped_objects/lemak_03_57.jpg

Proses selesai. Total 4 objek unik telah di-crop dan disimpan di direktori 'cropped_objects'.


# **INFERENCE OCR**

In [35]:

# --- Konfigurasi ---
# Direktori tempat gambar hasil crop Anda berada
INPUT_CROP_DIR = "cropped_objects" 
# Faktor penskalaan untuk memperbesar gambar (misalnya 3x)
UPSCALE_FACTOR = 5 

# Opsional: Tentukan path Tesseract jika tidak terdeteksi oleh sistem
# Contoh Windows:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# --- Proses Utama OCR (Tanpa Preprocessing CV2) ---
print(f"Memulai proses OCR hanya dengan upscaling {UPSCALE_FACTOR}x pada gambar di '{INPUT_CROP_DIR}'...")

extracted_data = {}

for filename in os.listdir(INPUT_CROP_DIR):
    if filename.endswith((".jpg", ".jpeg", ".png")):
        
        file_path = os.path.join(INPUT_CROP_DIR, filename)
        
        # Dapatkan label objek (misalnya 'garam_01.jpg' -> 'garam')
        try:
            object_label = filename.split('_')[0] 
        except IndexError:
            object_label = "unknown"

        # 1. Muat Gambar Asli (Hasil Crop)
        img_cv2 = cv2.imread(file_path)
        
        if img_cv2 is None:
            print(f"Gagal memuat {filename}.")
            continue
            
        # 2. Upscaling (Penskalaan)
        # Penting untuk teks kecil. INTER_CUBIC menjaga ketajaman tepi.
        upscaled_img = cv2.resize(
            img_cv2, 
            None, 
            fx=UPSCALE_FACTOR, 
            fy=UPSCALE_FACTOR, 
            interpolation=cv2.INTER_CUBIC
        )
        
        # 3. Konversi cv2 (NumPy array BGR) ke PIL Image (Pytesseract)
        # Pytesseract akan menangani konversi grayscale/biner secara internal.
        img_pil = Image.fromarray(cv2.cvtColor(upscaled_img, cv2.COLOR_BGR2RGB)) # Konversi BGR ke RGB untuk PIL

        # 4. Terapkan OCR dengan Konfigurasi
        # --psm 7: Asumsi satu baris teks (ideal untuk label nutrisi)
        # --oem 3: Gunakan mesin Tesseract terbaru
        custom_config = r'--oem 3 --psm 7' 
        
        text = pytesseract.image_to_string(img_pil, config=custom_config)
        
        # Bersihkan teks
        cleaned_text = text.strip().replace('\n', ' ').replace('|', 'l') 

        extracted_data[object_label] = cleaned_text
        
        print(f"\n--- {filename} ({object_label}) ---")
        print(f"Teks Terekstrak: {cleaned_text}")

# --- Ringkasan Hasil ---
print("\n----------------------------------------------------")
print("✅ Ringkasan Data Terekstrak Akhir:")
for key, value in extracted_data.items():
    print(f"{key:<10}: {value}")
print("----------------------------------------------------")

Memulai proses OCR hanya dengan upscaling 5x pada gambar di 'cropped_objects'...

--- serving_01_68.jpg (serving) ---
Teks Terekstrak: 

--- takaran-satuan_00_88.jpg (takaran-satuan) ---
Teks Terekstrak: 

--- lemak_03_57.jpg (lemak) ---
Teks Terekstrak: 

--- kalori_02_66.jpg (kalori) ---
Teks Terekstrak: _ Energi Total 160 kkal

----------------------------------------------------
✅ Ringkasan Data Terekstrak Akhir:
serving   : 
takaran-satuan: 
lemak     : 
kalori    : _ Energi Total 160 kkal
----------------------------------------------------


# **EXTRACT VALUE**

## base value

In [ ]:
def clean_and_transform_data(extracted_data: dict) -> dict:
    
    REQUIRED_KEYS = [
        "garam",
        "gula",
        "kalori",
        "karbo",
        "lemak",
        "protein",
        "serving",
        "takaran-satuan",
        "serat"
    ]

    # Buat dictionary hasil dengan semua value = 0 (kecuali serving = 1)
    cleaned_data = {
        key: (1 if key == "serving" else 0)
        for key in REQUIRED_KEYS
    }

    # 2. PROSES: Ekstraksi dan Pembatasan
    for key, value in extracted_data.items():
        
        # Hanya proses jika kunci tersebut termasuk dalam daftar wajib
        if key in REQUIRED_KEYS:
            
            # --- LANGKAH 1: EKSTRAKSI TOTAL (Mengambil SEMUA digit) ---
            # Regex: Hapus semua karakter non-digit (\D)
            # Ini menangani kasus seperti "10.5g" menjadi "105"
            all_digits_string = re.sub(r'\D', '', value)
            
            # --- LANGKAH 2: PEMBATASAN DAN KONVERSI ---
            
            if all_digits_string:
                # Ambil hanya 3 karakter pertama (slicing)
                max_3_digits_string = all_digits_string[:3]
                
                try:
                    # Konversi ke Integer
                    integer_value = int(max_3_digits_string)
                    
                    # Hanya update jika nilai baru lebih besar atau kunci bukan 'serving'
                    # Ini mencegah nilai inisiasi 'serving' 1 tertimpa oleh 0
                    if integer_value > 0 or key != "serving":
                        cleaned_data[key] = integer_value
                        
                except ValueError:
                    # Biarkan nilai tetap jika konversi gagal
                    pass
    
    return cleaned_data

In [ ]:
# Panggil fungsi
final_cleaned_data = clean_and_transform_data(extracted_data)

# Mencetak hasil akhir
print("\n--- Hasil Akhir Setelah Transformasi ---")
for key, value in final_cleaned_data.items():
    print(f"{key:<15}: {value}")

Data Awal yang Diterima: {'garam': 0, 'gula': 0, 'kalori': 160, 'karbo': 0, 'lemak': 0, 'protein': 0, 'serving': 1, 'takaran-satuan': 0, 'serat': 0}

--- Hasil Akhir Setelah Transformasi ---
garam          : 0
gula           : 0
kalori         : 160
karbo          : 0
lemak          : 0
protein        : 0
serving        : 1
takaran-satuan : 0
serat          : 0


## Total Nutrition

In [ ]:
def calculate_total_nutrients(cleaned_data: dict) -> dict:
    
    # 1. Pastikan nilai 'serving' ada dan bukan nol.
    # Jika serving tidak ada (misalnya karena gagal dideteksi), diasumsikan 1.
    serving_value = cleaned_data.get('serving', 1)
    if serving_value == 0:
        serving_value = 1 # Menghindari perkalian dengan nol yang akan menghasilkan nol

    # Kunci yang ingin dihitung totalnya (hanya nutrisi)
    NUTRIENT_KEYS = [
        "garam",
        "gula",
        "kalori",
        "karbo",
        "lemak",
        "protein",
        "serat"
    ]
    
    final_nutrients = {}

    # Kalkulasi
    for key in NUTRIENT_KEYS:
        # Ambil nilai nutrisi, jika tidak terdeteksi, value = 0
        nutrient_value = cleaned_data.get(key, 0)
        
        # Hitung Total Nutrisi per Kemasan
        total_value = nutrient_value * serving_value
        
        # Simpan hasil dalam dictionary final
        final_nutrients[key] = total_value
        
        
    return final_nutrients

In [46]:
# Panggil fungsi kalkulasi
total_nutrients = calculate_total_nutrients(final_cleaned_data)

# Mencetak hasil
print("\n--- Total Nutrisi Per Kemasan ---")
for key, value in total_nutrients.items():
    print(f"{key:<10}: {value}")


--- Total Nutrisi Per Kemasan ---
garam     : 0
gula      : 0
kalori    : 160
karbo     : 0
lemak     : 0
protein   : 0
serat     : 0


## Nutri-Score

In [ ]:
def calculate_nutri_score_simplified(cleaned_data: dict) -> str:
    """
    Menghitung Nutri-Score (A hingga E) berdasarkan data nutrisi per porsi.
    ASUMSI: Semua satuan adalah Gram (g), kecuali Kalori (Kkal).
    
    Args:
        cleaned_data (dict): Dictionary yang sudah dibersihkan (nilai per porsi/sajian).
                             Harus berisi kunci: 'kalori', 'gula', 'lemak', 'garam', 
                             'protein', 'serat', dan 'takaran-satuan'.
                             
    Returns:
        str: Huruf Nutri-Score (A, B, C, D, atau E).
    """
    
    # 1. NORMALISASI: Konversi Nilai Per Porsi ke Nilai Per 100g/ml
    
    # Asumsi: cleaned_data['takaran-satuan'] adalah berat per porsi dalam gram.
    serving_weight_g = cleaned_data.get('takaran-satuan', 100)
    if serving_weight_g <= 0:
        serving_weight_g = 100 # Default jika takaran satuan gagal terdeteksi
        
    normalization_factor = 100 / serving_weight_g

    # Nutrisi yang dinormalisasi (per 100g)
    norm_data = {}
    for key in ["kalori", "gula", "lemak", "garam", "protein", "serat"]:
        value = cleaned_data.get(key, 0)
        norm_data[key] = value * normalization_factor
    
    # ----------------------------------------------------------------------
    # 2. PERHITUNGAN POIN A (Elemen Negatif)
    # Thresholds diubah untuk menerima Kkal dan Gram secara langsung.

    A_score = 0
    
    # Ambil nilai per 100g
    energi_kkals = norm_data.get('kalori', 0)
    gula_g = norm_data.get('gula', 0)
    lemak_g = norm_data.get('lemak', 0)
    garam_g = norm_data.get('garam', 0)
    
    # Thresholds Poin Energi (Kkal/100g)
    if energi_kkals > 800: A_score += 10
    elif energi_kkals > 720: A_score += 9
    elif energi_kkals > 640: A_score += 8
    elif energi_kkals > 560: A_score += 7
    elif energi_kkals > 480: A_score += 6
    elif energi_kkals > 400: A_score += 5
    elif energi_kkals > 320: A_score += 4
    elif energi_kkals > 240: A_score += 3
    elif energi_kkals > 160: A_score += 2
    elif energi_kkals > 80: A_score += 1

    # Thresholds Poin Gula (g/100g)
    if gula_g > 45: A_score += 10
    elif gula_g > 40: A_score += 9
    elif gula_g > 36: A_score += 8
    elif gula_g > 31: A_score += 7
    elif gula_g > 27: A_score += 6
    elif gula_g > 22.5: A_score += 5
    elif gula_g > 18: A_score += 4
    elif gula_g > 13.5: A_score += 3
    elif gula_g > 9: A_score += 2
    elif gula_g > 4.5: A_score += 1

    # Thresholds Poin Lemak Jenuh (g/100g) - Menggunakan ambang yang sama seperti lemak jenuh
    if lemak_g > 10: A_score += 10
    elif lemak_g > 9: A_score += 9
    elif lemak_g > 8: A_score += 8
    elif lemak_g > 7: A_score += 7
    elif lemak_g > 6: A_score += 6
    elif lemak_g > 5: A_score += 5
    elif lemak_g > 4: A_score += 4
    elif lemak_g > 3: A_score += 3
    elif lemak_g > 2: A_score += 2
    elif lemak_g > 1: A_score += 1
        
    # Thresholds Poin Garam (g/100g)
    if garam_g > 1.5: A_score += 10
    elif garam_g > 1.35: A_score += 9
    elif garam_g > 1.2: A_score += 8
    elif garam_g > 1.05: A_score += 7
    elif garam_g > 0.9: A_score += 6
    elif garam_g > 0.75: A_score += 5
    elif garam_g > 0.6: A_score += 4
    elif garam_g > 0.45: A_score += 3
    elif garam_g > 0.3: A_score += 2
    elif garam_g > 0.15: A_score += 1
        
    # ----------------------------------------------------------------------
    # 3. PERHITUNGAN POIN C (Elemen Positif)
    
    C_score = 0
    serat_g = norm_data.get('serat', 0)
    protein_g = norm_data.get('protein', 0)
    
    # Thresholds Poin Serat (g/100g)
    if serat_g >= 4.7: C_score += 5
    elif serat_g >= 3.7: C_score += 4
    elif serat_g >= 2.8: C_score += 3
    elif serat_g >= 1.9: C_score += 2
    elif serat_g >= 0.9: C_score += 1

    # Thresholds Poin Protein (g/100g)
    if protein_g >= 8.0: C_score += 5
    elif protein_g >= 6.4: C_score += 4
    elif protein_g >= 4.8: C_score += 3
    elif protein_g >= 3.2: C_score += 2
    elif protein_g >= 1.6: C_score += 1
        
    # ----------------------------------------------------------------------
    # 4. KALKULASI AKHIR

    final_score = A_score - C_score

    # Konversi Skor ke Huruf (Thresholds Umum)
    if final_score <= -1:
        return "A"
    elif final_score <= 2:
        return "B"
    elif final_score <= 10:
        return "C"
    elif final_score <= 18:
        return "D"
    else:
        return "E"

In [54]:


nutri_score = calculate_nutri_score_simplified(final_cleaned_data)

# print(f"Data Per Porsi: {contoh_data_bersih}")
# print(f"Berat Sajian: {contoh_data_bersih['takaran-satuan']} g")
print(f"\nNutri-Score Hasil Simplifikasi: {nutri_score}")


Nutri-Score Hasil Simplifikasi: B


# **INISIASI VARIABEL UNTUK DATABASE**

In [55]:
total_nutrients

{'garam': 0,
 'gula': 0,
 'kalori': 160,
 'karbo': 0,
 'lemak': 0,
 'protein': 0,
 'serat': 0}

In [ ]:
# variabel-variabel yang siap masuk dalam database

garam, gula, kalori, karbo, lemak, protein, serat = total_nutrients.values()
nutri_score

In [61]:
print(f'nutri-score: {nutri_score}\ngaram: {garam}\ngula: {gula}\nkalori: {kalori}\nkarbohidrat: {karbo}\nlemak: {lemak}\nprotein: {protein}\nserat: {serat}')

nutri-score: B
garam: 0
gula: 0
kalori: 160
karbohidrat: 0
lemak: 0
protein: 0
serat: 0


# **Menyimpan library yang dibutuhkan** 

In [58]:
!pip freeze | grep -E 'ultralytics|torch|opencv-python|pandas|scikit-learn|pytesseract|Pillow|numpy|matplotlib|seaborn|tqdm' > requirements.txt